# CircleViz examples


### Requirements

These examples require the installation of the following python modules

* Pandas

```
pip install pandas
```


In [36]:
# Jenks natural breaks classification for example.
# Slow version - does not require C bindings
# For fast version, see pip install -e "git+https://github.com/perrygeo/jenks.git#egg=jenks"
import json
from pprint import pprint as pp

def jenks_matrices_init(data, n_classes):
    #fill the matrices with data+1 arrays of n_classes 0s
    lower_class_limits = []
    variance_combinations = []
    for i in xrange(0, len(data)+1):
        temp1 = []
        temp2 = []
        for j in xrange(0, n_classes+1):
            temp1.append(0.)
            temp2.append(0.)
        lower_class_limits.append(temp1)
        variance_combinations.append(temp2)

    inf = float('inf')
    for i in xrange(1, n_classes+1):
        lower_class_limits[1][i] = 1.
        variance_combinations[1][i] = 0.
        for j in xrange(2, len(data)+1):
            variance_combinations[j][i] = inf

    return lower_class_limits, variance_combinations

def jenks_matrices(data, n_classes):
    lower_class_limits, variance_combinations = jenks_matrices_init(data, n_classes)

    variance = 0.0
    for l in xrange(2, len(data)+1):
        sum = 0.0
        sum_squares = 0.0
        w = 0.0
        for m in xrange(1, l+1):
            # `III` originally
            lower_class_limit = l - m + 1
            val = data[lower_class_limit-1]

            # here we're estimating variance for each potential classing
            # of the data, for each potential number of classes. `w`
            # is the number of data points considered so far.
            w += 1

            # increase the current sum and sum-of-squares
            sum += val
            sum_squares += val * val

            # the variance at this point in the sequence is the difference
            # between the sum of squares and the total x 2, over the number
            # of samples.
            variance = sum_squares - (sum * sum) / w

            i4 = lower_class_limit - 1

            if i4 != 0:
                for j in xrange(2, n_classes+1):
                    if variance_combinations[l][j] >= (variance + variance_combinations[i4][j - 1]):
                        lower_class_limits[l][j] = lower_class_limit
                        variance_combinations[l][j] = variance + variance_combinations[i4][j - 1]

        lower_class_limits[l][1] = 1.
        variance_combinations[l][1] = variance

    return lower_class_limits, variance_combinations

def get_jenks_breaks(data, lower_class_limits, n_classes):
    k = len(data) - 1
    kclass = [0.] * (n_classes+1)
    countNum = n_classes

    kclass[n_classes] = data[len(data) - 1]
    kclass[0] = data[0]

    while countNum > 1:
        elt = int(lower_class_limits[k][countNum] - 2)
        kclass[countNum - 1] = data[elt]
        k = int(lower_class_limits[k][countNum] - 1)
        countNum -= 1

    return kclass

def jenks(data, n_classes):
    if n_classes > len(data): return

    data.sort()

    lower_class_limits, _ = jenks_matrices(data, n_classes)

    return get_jenks_breaks(data, lower_class_limits, n_classes)

In [37]:
import pandas as pd
import sys
import os
sys.path.append("..") # Adds higher directory to python modules path.  Use to run this example locallay without a global pip install.

from mapboxgl.viz import *
from mapboxgl.utils import df_to_geojson, create_radius_stops, scale_between
from mapboxgl.colors import create_color_stops

In [38]:
df = pd.read_csv('../examples/points.csv')
df['Avg Total Payments'] = df['Avg Total Payments'].round(2)
df['Avg Covered Charges'] = df['Avg Covered Charges'].round(2)
df['Total Discharges'] = df['Total Discharges'].round(2)
df['Avg Medicare Payments'] = df['Avg Medicare Payments'].round(2)
df.head(5)

,Avg Total Payments,Avg Covered Charges,Total Discharges,Avg Medicare Payments,admin1_id,Provider Id,admin2_id,lon,lat
0,8749.03,35247.03,58.75,7678.21,USA101,10001,USA201069,-85.362856,31.216215
1,6812.13,16451.09,28.96,5793.63,USA101,10005,USA201119,-88.142797,32.452976
2,8197.24,36942.36,45.36,7145.96,USA101,10006,USA201077,-87.682872,34.794052
3,4860.83,12079.54,27.41,4047.03,USA101,10007,USA201039,-86.254597,31.291993
4,5898.14,16148.75,17.89,4963.55,USA101,10008,USA201041,-86.265102,31.693554


In [21]:
data = df_to_geojson(df, 
                     properties=['Avg Medicare Payments', 'Avg Covered Charges'],
                     precision=4)

Set the access token. Alternatively, set this as a `MAPBOX_ACCESS_TOKEN` environment variable


In [22]:
# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

Create the visualization

In [23]:
viz = CircleViz(data, access_token=token, height='400px')

Generate data breaks and color stops from colorBrewer

In [24]:
breaks = jenks(df['Avg Medicare Payments'].tolist(), 8)
color_stops = create_color_stops(breaks, colors='YlGnBu')
color_stops

[[2610.4, 'rgb(255,255,217)'],
 [5546.98, 'rgb(237,248,177)'],
 [7058.17, 'rgb(199,233,180)'],
 [8620.55, 'rgb(127,205,187)'],
 [10533.71, 'rgb(65,182,196)'],
 [13176.35, 'rgb(29,145,192)'],
 [17089.87, 'rgb(34,94,168)'],
 [25098.25, 'rgb(37,52,148)'],
 [41836.88, 'rgb(8,29,88)']]

In [25]:
# Test the circle viz on the Avg Medicare Payments property
viz.color_property = "Avg Medicare Payments"
viz.color_stops = color_stops
viz.center = (-95, 40)
viz.zoom = 3

viz.show()

In [26]:
# Add labels to the viz
viz.label_property = "Avg Medicare Payments"
viz.show()

In [27]:
# Generate data breaks and color stops from colorBrewer
breaks = jenks(df['Avg Covered Charges'].tolist(), 8)
color_stops = create_color_stops(breaks, colors='YlOrRd')

You can also create a vizualization by intializing the object with all of the relevant
attributes.

In [28]:
viz2 = CircleViz(data, access_token=token, color_property='Avg Covered Charges',
                 color_stops=color_stops, center=[-95, 40], zoom=3)
viz2.show()

Using a different Mapbox style

In [29]:
viz3 = CircleViz(data, access_token=token, color_property='Avg Covered Charges',
                 style_url='mapbox://styles/mapbox/dark-v9?optimize=true',
                 color_stops=color_stops, center=[-95, 40], zoom=3)
viz3.show()

Create a graduated cricle viz based on two data properties

In [30]:
# Generate data breaks and color stops from colorBrewer
color_breaks = jenks(df['Avg Covered Charges'].tolist(), 8)
color_stops = create_color_stops(color_breaks, colors='Spectral')

# Generate radius breaks from data domain and circle-radius range
radius_breaks = jenks(df["Avg Medicare Payments"].tolist(), 8)
radius_stops = create_radius_stops(radius_breaks, 1, 10)

In [31]:
viz4 = GraduatedCircleViz(data, access_token=token)
viz4.color_property = "Avg Covered Charges"
viz4.color_stops = color_stops
viz4.radius_property = "Avg Medicare Payments"
viz4.radius_stops = radius_stops
viz4.center = (-95, 40)
viz4.zoom = 3

viz4.show()

Swap the map to a satellite image with a larger radius scale

In [32]:
radius_stops = create_radius_stops(radius_breaks, 1, 15)
color_stops = create_color_stops(color_breaks, colors='RdBu')

viz5 = GraduatedCircleViz(data, access_token=token, style_url='mapbox://styles/mapbox/satellite-streets-v10?optimize=true')
viz5.color_property = "Avg Covered Charges"
viz5.color_stops = color_stops
viz5.radius_property = "Avg Medicare Payments"
viz5.radius_stops = radius_stops
viz5.center = (-95, 40)
viz5.zoom = 3

viz5.show()